# Trying my best two models

In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
'''knn_Simasymmetric_K6_Shr16_AsymAlpha0.024628530649850728.csv
Complete · Luca Pagano · 25d ago
0.08603'''

In [ ]:
'''
result,topK,l1_ratio,alpha

0.06001721416196575,142.0,0.4033590645217344,9.521340863590419e-05
'''

In [15]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender_Peppe import MultiThreadSLIM_SLIMElasticNetRecommender

slim_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)

topK = int(142.0)
alpha = 9.521340863590419e-05
l1_ratio = 0.4033590645217344
slim_recommender.fit(topK=topK, l1_ratio=l1_ratio, alpha=alpha)

knn_recommender = ItemKNNCFRecommender(URM_train_validation)
knn_recommender.fit(shrink=16, topK=6, asymmetric_alpha=0.024628530649850728)

Progress: 100%|██████████| 38121/38121 [09:54<00:00, 64.16items/s] 


Similarity column 38121 (100.0%), 4775.37 column/sec. Elapsed time 7.98 sec


In [16]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective_function(trial):

    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


    recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)

    new_similarity = slim_recommender.W_sparse * alpha + knn_recommender.W_sparse * (1-alpha)

    recommender.fit(new_similarity)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [17]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/hybrid_slim_KNN_MERGINGMODELS.csv", index = False)

In [18]:
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_knn", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-23 19:43:37,635] A new study created in memory with name: hybrid_slim_knn
<ipython-input-16-3192f54068d0>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 16.13 sec. Users per second: 2207


[I 2024-12-23 19:43:53,809] Trial 0 finished with value: 0.05952740821008664 and parameters: {'alpha': 0.7551383802895948}. Best is trial 0 with value: 0.05952740821008664.
<ipython-input-16-3192f54068d0>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.91 sec. Users per second: 2237


[I 2024-12-23 19:44:09,759] Trial 1 finished with value: 0.05941037063792883 and parameters: {'alpha': 0.7429532102635861}. Best is trial 0 with value: 0.05952740821008664.
<ipython-input-16-3192f54068d0>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


...

[I 2024-12-23 20:09:11,419] Trial 97 finished with value: 0.06002086858997755 and parameters: {'alpha': 0.9999307173872467}. Best is trial 68 with value: 0.06028501639921608.
<ipython-input-16-3192f54068d0>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.26 sec. Users per second: 2333


[I 2024-12-23 20:09:26,718] Trial 98 finished with value: 0.06018108816781188 and parameters: {'alpha': 0.8724181291175191}. Best is trial 68 with value: 0.06028501639921608.
<ipython-input-16-3192f54068d0>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.76 sec. Users per second: 2259


[I 2024-12-23 20:09:42,513] Trial 99 finished with value: 0.05248409799842431 and parameters: {'alpha': 0.17779817360350747}. Best is trial 68 with value: 0.06028501639921608.


In [19]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.06028501639921608, 'alpha': 0.9242014366593713}


In [20]:
# let's see the performance on test of the slimnet

result_dict, _ = evaluator_test.evaluateRecommender(slim_recommender)

MAP = result_dict.loc[10]["MAP"]

print("MAP on test considering validation: "+str(MAP))

EvaluatorHoldout: Processed 35595 (100.0%) in 15.44 sec. Users per second: 2305
MAP on test considering validation: 0.06001747614811059


So we had slightly better results than with the slimnet

# Submission

In [22]:
# Let's produce the csv

alpha = best_hyperparams["alpha"]

slim_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)

topK = int(142.0)
alpha = 9.521340863590419e-05
l1_ratio = 0.4033590645217344
slim_recommender.fit(topK=topK, l1_ratio=l1_ratio, alpha=alpha)

knn_recommender = ItemKNNCFRecommender(URM_all)
knn_recommender.fit(shrink=16, topK=6, asymmetric_alpha=0.024628530649850728)

recommender = ItemKNNCustomSimilarityRecommender(URM_all)

new_similarity = slim_recommender.W_sparse * alpha + knn_recommender.W_sparse * (1-alpha)

recommender.fit(new_similarity)

Progress: 100%|██████████| 38121/38121 [13:22<00:00, 47.52items/s] 


Similarity column 38121 (100.0%), 4353.27 column/sec. Elapsed time 8.76 sec


In [23]:
# Let's produce the csv
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv("submissions/hybrid_slim_knn.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
